In [1]:
# use torch_2022

import numpy as np
#import pandas as pd
import pickle
import time
import os
import copy
import sys
#import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, models, transforms
from torchvision.io import read_image


# models - you do need the weights for the transformations.
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from torchvision.models import regnet_x_8gf, RegNet_X_8GF_Weights
from torchvision.models import swin_t, Swin_T_Weights
from torchvision.models import wide_resnet50_2, Wide_ResNet50_2_Weights

#import wandb

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)
print(sys.version)
print(torch.__version__)
print(torchvision.__version__)

cpu
3.8.13 (default, Mar 28 2022, 11:38:47) 
[GCC 7.5.0]
1.12.1
0.13.1


Make a loop: 

pseudo python code


For each model:
    For each fature:
        feature_list = []
        img_list = []
        for img:
            score = model(img)

            feature_list.add(score)
            img_list.add(img)

        df = pd.Dataframe({feature : feature_list, 'img_id' : img_list})
        df.to_pkl(.../model_feature.pkl)





In [3]:
# NOW YOU DO NOT NEED TRAIN; TEST OR VAL AS EVERYTHING IS ALL IMAGES AND NO EXTRA TRANSFORAMTIONS:
# ALSO you do not have labels

class CustomImageDataset(Dataset):
    def __init__(self, img_dir, attribute_dict, transform=None):


        self.img_id = attribute_dict['img'] # img_id + .jpg
        self.img_dir = img_dir
        self.transform = transform


        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_id)

    def __getitem__(self, idx):
        img_id = self.img_id[idx]
        img_path = os.path.join(self.img_dir, img_id)
        image = read_image(img_path)
        
        if self.transform:
            image = self.transform(image)

        return image, img_id

In [4]:
def change_head(model_name, model, num_classes):

    if model_name == 'convnext_tiny':
        model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes, bias=False).to(device)
        #print(f'new head: {model.classifier[2]}')

    elif model_name == 'efficientnet_v2_s':
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes, bias=False).to(device)
        #print(f'new head: {model.classifier[1]}')

    elif model_name == 'regnet_x_8gf':
        model.fc = nn.Linear(model.fc.in_features, num_classes, bias=False).to(device)
        #print(f'new head: {model.fc}')

    elif model_name == 'swin_t':
        model.head = nn.Linear(model.head.in_features, num_classes, bias=False).to(device)
        #print(f'new head: {model.head}')

    elif model_name == 'wide_resnet50_2':
        model.fc = nn.Linear(model.fc.in_features, num_classes, bias=False).to(device)
        #print(f'new head: {model.fc}')

    elif model_name == 'squeezenet1_1' :
        model.classifier[1] = nn.Conv2d(model.classifier[1].in_channels, num_classes, kernel_size=(1, 1), stride=(1, 1), bias=False).to(device)
        #print(f'new head: {model.classifier[1]}')

    elif model_name == 'shufflenet_v2_x0_5' :
        model.fc = nn.Linear(model.fc.in_features, num_classes, bias=False).to(device)
        #print(f'new head: {model.fc}')

    elif model_name == 'mnasnet0_5' :
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes, bias=False).to(device)
        #print(f'new head: {model.classifier[1]}')    

    elif model_name == 'mobilenet_v3_small' :
        model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes, bias=False).to(device)
        #print(f'new head: {model.classifier[3]}')

    else:
        print('Unddefined model name...')


In [5]:
# JUST LOAD PRETRAINED MODEL

def get_model(hyperparameters):

    weight_dict = {'convnext_tiny': ConvNeXt_Tiny_Weights.DEFAULT,
                'efficientnet_v2_s' : EfficientNet_V2_S_Weights.DEFAULT,
                'regnet_x_8gf' : RegNet_X_8GF_Weights.DEFAULT,
                'swin_t' : Swin_T_Weights.DEFAULT,
                'wide_resnet50_2' : Wide_ResNet50_2_Weights.DEFAULT}

    model_dict = {'convnext_tiny': convnext_tiny(weights = weight_dict['convnext_tiny']).to(device),
                'efficientnet_v2_s' : efficientnet_v2_s(weights = weight_dict['efficientnet_v2_s']).to(device),
                'regnet_x_8gf' : regnet_x_8gf(weights = weight_dict['regnet_x_8gf']).to(device),
                'swin_t' : swin_t(weights = weight_dict['swin_t']).to(device),
                'wide_resnet50_2' : wide_resnet50_2(weights = weight_dict['wide_resnet50_2']).to(device)}

    # model_name = hyperparameters['model_name']

    model_name = hyperparameters['model_name']
    attribute = hyperparameters['attribute']

    pt_model_dir = "/home/simon/Documents/computerome/done_RA_models/"
    pt_model_name = f'{model_name}_{attribute}'

    model = model_dict[model_name]
    change_head(model_name, model, 1)
    
    model.load_state_dict(torch.load(f'{pt_model_dir}{pt_model_name}_SD.pth', map_location=torch.device('cpu'))) #!!!! Remove or change map_location
    model.eval()

    return(model, weight_dict)

In [6]:
# data loader
def make_loader(batch_size, weights, attribute_dict):

    data_transform = transforms.Compose([weights.transforms()])

    # img_dir = '/media/simon/Seagate Expansion Drive/images_spanner' #local
    img_dir = '/home/projects/ku_00017/data/raw/bodies/images_spanner' # computerome

    image_dataset = CustomImageDataset(img_dir, attribute_dict, transform=data_transform)
    dataloader = DataLoader(image_dataset, batch_size=batch_size, shuffle=False)

    dataset_size = len(image_dataset)

    return dataloader, dataset_size

In [7]:
def make(hyperparameters):

    # Make the model
    model, weight_dict = get_model(hyperparameters)

    model_name = hyperparameters['model_name']

    #Choose model and wieghts
    weights = weight_dict[model_name] # you need these later right as they hold the appropiate data transforamtions
    model = model.to(device)
    # wandb.watch(model)
    
    # is this an ok place? 
    #dict_dir = '/home/simon/Documents/Bodies/data/RA/dfs/' # change to computerome location
    dict_dir = '/home/projects/ku_00017/data/raw/bodies/RA_annotations/' # computerome

    with open(f'{dict_dir}ra_ens_annotated_dict.pkl', 'rb') as file:
        attribute_dict = pickle.load(file)

    # Make the data
    dataloader, dataset_size = make_loader(batch_size=hyperparameters['batch_size'],  weights = weights, attribute_dict = attribute_dict)

    return model, dataloader, dataset_size

In [8]:
def predict(model, dataloader):
    model.eval() # one more time why not.

    image_list = []
    score_list = []

    count = 0

    # Run the model on some test examples
    with torch.no_grad():
        
        for images, img_id in dataloader:
                
            images = images.to(device)
            outputs = model(images)

            image_list += list(img_id)
            score_list += list(outputs.squeeze().detach().cpu().numpy())
            
    return(image_list, score_list)

In [ ]:
def the_loop():
    relevant_models = ['convnext_tiny', 'efficientnet_v2_s', 'swin_t']

    attributes = ['all_negative_emotions_t1', 'all_mass_protest', 'all_militarized',
                'all_urban', 'all_negative_emotions_t2', 'all_privat', 'all_public', 
                'all_rural', 'all_formal', 'all_damaged_property']


    data_dir = '/home/projects/ku_00017/data/generated/bodies/ra_outputs/'

    for model_name in relevant_models:
        
        score_dict = {}
        
        for attribute in attributes:

            hyperparameters = {"model_name" : model_name, "attribute" : attribute, "batch_size": 32}
            model, dataloader, dataset_size = make(hyperparameters)
            image_list, score_list = predict(model, dataloader)

            score_dict['attribute_score'] = score_list,
            score_dict['attribute_id'] = image_list # not sure you get the rigth order so this is just for debug really.

        dict_name = f'{model_name}_score_dict.pkl'

        with open(f'{data_dir}{dict_name}', 'wb') as file:
            pickle.dump(score_dict, file)